In [1]:
'''
This file prepares a dataset for MOFA, by converting to long format and building the columns sample, group, feature,
view, and value.
'''

'\nThis file prepares a dataset for MOFA, by converting to long format and building the columns sample, group, feature,\nview, and value.\n'

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
neg = pd.read_csv('metabolites_neg_cleaned.csv')
pos = pd.read_csv('metabolites_pos_cleaned.csv')
genes = pd.read_csv('rna_vst_proc.csv')

/var/folders/v1/ymptnwwx3x1b7_mw0nns09wr0000gn/T/ipykernel_45789/3826854301.py:3: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160) have mixed types. Specify dtype option on import or set low_memory=False.
  genes = pd.read_csv('rna_vst_proc.csv')


In [3]:
genes.head()

,Unnamed: 0,treatment,control,control.1,control.2,control.3,control.4,control.5,low,low.1,...,low.49,low.50,low.51,low.52,high.46,high.47,high.48,high.49,high.50,high.51
0,NaN,time_point,1H,1H,1H,1H,1H,1H,1H,1H,...,7D,7D,7D,7D,7D,7D,7D,7D,7D,7D
1,KEGG,gene,C01_1H,C02_1H,C03_1H,C04_1H,C05_1H,C06_1H,L01_1H,L02_1H,...,L02_7D,L03_7D,L05_7D,L06_7D,H01_7D,H02_7D,H03_7D,H04_7D,H05_7D,H06_7D
2,K06768,Dapma7bEVm000001,6.722429721,6.659031761,7.109533697,7.057990459,6.952206768,7.047009312,6.962151921,6.608657123,...,5.937831215,6.035896015,6.16231173,6.213557336,6.567493029,6.126517438,5.207510668,6.279863974,6.416723012,6.291636638
3,NaN,Dapma7bEVm000002,9.814267638,9.493683803,9.356571886,9.737901716,9.956902675,10.14702089,9.199138445,9.473363735,...,8.865364162,9.412111581,9.146006017,9.005020605,9.870256252,9.312805307,9.808536766,8.718030232,9.926635484,9.733115355
4,K17307,Dapma7bEVm000003,8.236806072,7.809384967,7.070101378,8.276224578,6.227475261,7.844540904,6.741203908,7.172462395,...,7.212206142,6.987310672,6.513786365,7.373906991,6.689082618,6.892385835,7.178607897,6.899900105,7.17709791,6.697307499


In [4]:
#to process transcriptomics
#delete 1st column
genes_removed = genes.iloc[:,1:]
#set 2nd row as column names
col_name_list = genes_removed.iloc[1].tolist()
genes_removed.columns = col_name_list
#delete 1st 2 rows
genes_rows_removed = genes_removed.iloc[2:,:]
genes_rows_removed.to_csv('transcriptome_cleaned.csv')

In [5]:
#to process both
def mofa_melting(df,idvars,view):
    #melt so column 1 is 1st column, column 2 is second column, values are third column
    #Dapma/mz is sample, group is column name
    melted_df = pd.melt(df, id_vars=[idvars], var_name='sample', value_name='value')
    #feature is control, low, high based on column names
    melted_df['group'] = melted_df['sample'].apply(lambda x: 'control' if x.startswith('C') else ('low' if x.startswith('L') else 'high'))
    #view is either transcriptomic or metabolomic
    melted_df['view'] = view
    #rename first column
    melted_renamed = melted_df.rename(columns={idvars:'feature'})
    #reorder columns
    melted_reordered = melted_renamed[['sample','group','feature','view','value']]
    return melted_reordered

In [6]:
processed_genes = mofa_melting(genes_rows_removed,'gene','transcriptomic')
processed_neg = mofa_melting(neg,'mz','metabolomic')
processed_pos = mofa_melting(pos,'mz','metabolomic')

In [9]:
#combine dfs into one
neg_omics = pd.concat([processed_genes, processed_neg], ignore_index=True)
pos_omics = pd.concat([processed_genes, processed_pos], ignore_index=True)

In [10]:
#save to csv files
neg_omics.to_csv('mofa_neg_omics.csv',index=False)
pos_omics.to_csv('mofa_pos_omics.csv',index=False)

In [17]:
processed_genes.tail()

,sample,group,feature,view,value
2540179,H06_7D,high,Dapma7bEVm030753,transcriptomic,5.061786
2540180,H06_7D,high,Dapma7bEVm030754,transcriptomic,5.765166
2540181,H06_7D,high,Dapma7bEVm030755,transcriptomic,5.79141
2540182,H06_7D,high,Dapma7bEVm629765,transcriptomic,2.558036
2540183,H06_7D,high,Dapma7bEVm629948,transcriptomic,4.877498


In [18]:
processed_neg.tail()

,sample,group,feature,view,value
263355,H06_7D,high,608.99190,metabolomic,17.484499
263356,H06_7D,high,610.05793,metabolomic,15.532763
263357,H06_7D,high,611.04185,metabolomic,15.473740
263358,H06_7D,high,611.14577,metabolomic,18.462370
263359,H06_7D,high,611.27737,metabolomic,14.866806


In [52]:
processed_genes.to_csv('for_mofa_transcriptomic.csv')
processed_neg.to_csv('for_mofa_metabolomic_neg.csv')
processed_pos.to_csv('for_mofa_metabolomic_pos.csv')